In [1]:
from InstagramAPI import InstagramAPI

In [3]:
api = InstagramAPI("Your_Username", "Your_Passwords")
api.login()

user_id = api.username_id

Request return 405 error!
{'message': '', 'status': 'fail'}
Request return 404 error!
Login success!



# get id user

In [65]:
import requests
def get_id(username):
    url = "https://www.instagram.com/web/search/topsearch/?context=blended&query="+username+"&rank_token=0.3953592318270893&count=1"
    response = requests.get(url)
    respJSON = response.json()
    try:
        user_id=None
        for i in respJSON['users']:
            if i.get('user').get('username')==username:
                user_id = str(i.get("user").get("pk") )
                break
            elif username in i.get('user').get('username'):
                user_id = str(i.get("user").get("pk") )
                
        if not user_id:
            user_id=respJSON['users'][0].get('user').get('pk')
        return user_id
    except:
        return "Unexpected error"

# get user info

In [300]:
keys_info='''biography
follower_count
following_count
full_name
is_business
is_verified
linked_fb_user
media_count
pk
username
usertags_count
total_igtv_videos
total_ar_effects'''.split('\n')
keys_follow='''full_name
is_private
is_verified
pk
username'''.split('\n')


In [333]:
def user_info(user, following=False,maxfollowing=None, follower=False, maxfollower=None):
    user_id=None
    try:
        int(user)
        user_id=user
    except: 
        user_id=get_id(user)
        
    f=api.getUsernameInfo(user_id)
    if f:
        hasil=api.LastJson
        info=defaultdict(lambda : None)
        for i in keys_info:
            if i=='pk':
                info['user_id']=hasil['user'].get(i)
            else:
                info[i]=hasil['user'].get(i)
        hasil=info
        hasil={'user':hasil}
    else:
        return '{} not Found'.format(user)
    if following:
        fwr=getTotalFollowings(api,user_id, maxfollowing)
        hasil.update({'followings':fwr})
    if follower:
        fwr=getTotalFollowers(api,user_id, maxfollower)
        hasil.update({'followers':fwr})
    
    follower=[]
    for i in hasil['followers']:
        i.pop('profile_pic_id',None)
        i.pop('profile_pic_url',None)
        i.pop('latest_reel_media',None)
        i.pop('has_anonymous_profile_picture',None)
        follower.append(i)
        
    following=[]
    for i in hasil['followings']:
        i.pop('profile_pic_id',None)
        i.pop('profile_pic_url',None)
        i.pop('latest_reel_media',None)
        i.pop('has_anonymous_profile_picture',None)
        following.append(i)
    
    hasil.update({'followings':following})
    hasil.update({'followers':follower})
    return hasil
        
                                    

# get follower and following

In [332]:
def getTotalFollowings(api, user_id, limit=None):
    """
    Returns the list of followers of the user.
    It should be equivalent of calling api.getTotalFollowers from InstagramAPI
    """
    if limit:
        limit=int(limit/180)

    followings = []
    next_max_id = True
    index=0
    while next_max_id:
        
        if next_max_id is True:
            next_max_id = ''

        api.getUserFollowings(user_id, maxid=next_max_id)
        result=api.LastJson
        followings.extend(result.get('users', []))
        next_max_id =result.get('next_max_id', '')
        
        if index==limit:
            break
        index+=1
    return followings

In [215]:
def getTotalFollowers(api, user_id, limit=None):
    """
    Returns the list of followers of the user.
    It should be equivalent of calling api.getTotalFollowers from InstagramAPI
    """
    if limit:
        limit=int(limit/180)

    followers = []
    next_max_id = True
    index=0
    while next_max_id:
        
        if next_max_id is True:
            next_max_id = ''

        api.getUserFollowers(user_id, maxid=next_max_id)
        result=api.LastJson
        followers.extend(result.get('users', []))
        next_max_id =result.get('next_max_id', '')
        
        if index==limit:
            break
        index+=1
    return followers

### example

In [339]:
data=user_info('gojekindonesia', follower=True, maxfollower=None, following=True, maxfollowing=None)

Except on SendRequest (wait 60 sec and resend): ('Connection broken: OSError("(10054, \'WSAECONNRESET\')",)', OSError("(10054, 'WSAECONNRESET')",))


In [344]:
data.keys()

dict_keys(['user', 'followings', 'followers'])

In [346]:
import json
with open('gojek_profile.json','w') as f:
    json.dump(data,f)

In [347]:
data['user']

defaultdict(<function __main__.user_info.<locals>.<lambda>>,
            {'biography': 'Melayani kamu & menghidupi banyak orang melalui 18 layanan hanya dalam 1 aplikasi. #GOJEKSuperApp!',
             'follower_count': 710133,
             'following_count': 30,
             'full_name': 'GOJEK Indonesia',
             'is_business': True,
             'is_verified': True,
             'linked_fb_user': None,
             'media_count': 108,
             'total_ar_effects': 0,
             'total_igtv_videos': 5,
             'user_id': 1367852158,
             'username': 'gojekindonesia',
             'usertags_count': 88168})

# get post

In [369]:
p=api.getTotalUserFeed(get_id('gojekindonesia'))

In [382]:
keys_post='''caption
comment_count
device_timestamp
id
like_count
pk
taken_at
user'''.split('\n')

In [383]:
all_post=[]
for i in p:
    post={}
    for j in keys_post:    
        post.update({j:i.get(j)})
    all_post.append(post)

In [404]:
with open('gojek_post.json','w') as f:
    json.dump(all_post,f)

In [ ]:
keys_comment='''user
comment_like_count
created_at
pk
text'''.split('\n')
all_comment=[]
from tqdm import tqdm
for posting in tqdm(all_post):
    id_=posting['id']
    comments = []
    next_max_id = True
    index=0
    while next_max_id:
        if next_max_id is True:
            next_max_id = ''
        api.getMediaComments(id_,next_max_id)
        result=api.LastJson
        a=result.get('comments', [])
        a={'comments':a}
        a.update({'post_id':id_})
        comments.append(a)
        if index%5==0:
            print(len(comments))
        index+=1
        next_max_id =result.get('next_max_id', '')


  0%|                                                                                          | 0/108 [00:00<?, ?it/s]

1


  1%|▊                                                                                 | 1/108 [00:01<02:35,  1.46s/it]

1
6
11
16
21
26
31
36
41
46
51


  2%|█▌                                                                                | 2/108 [00:39<21:46, 12.33s/it]

...

1
6
11
16
21


 90%|████████████████████████████████████████████████████████████████████████▊        | 97/108 [17:54<01:36,  8.80s/it]

1
6
11
16
21
26
31
36


 91%|█████████████████████████████████████████████████████████████████████████▌       | 98/108 [18:11<01:52, 11.24s/it]

1
6
11
16
21
26
